# Install packages

In [1]:
!apt update
from google.colab import drive
import os

drive.mount('/content/drive')

!pip install opencv-python-headless
!pip install pydicom
import pydicom
import pandas as pd

!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install pillow
!pip install numpy
!pip install openpyxl

import pytesseract
import numpy as np
from PIL import Image
from PIL import ImageGrab

from typing import List
from PIL import Image

import ipywidgets as widgets

import os
import re
import time
import cv2

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,346 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [44.6 kB]
Get:13 http://secur

In [2]:
def replace_value(dataframe, row_index, column_name, new_value) -> pd.DataFrame:
    """
    Substitua o valor de uma coluna especificada em uma linha especificada em um DataFrame.

    Parâmetros:
    dataframe (pd.DataFrame): DataFrame original.
    row_index (int): índice da linha onde o valor será substituído.
    column_name (str): nome da coluna onde o valor será substituído.
    new_value: novo valor a ser inserido.

    Retorna:
    pd.DataFrame: DataFrame após a substituição do valor.
    """

    dataframe.at[row_index, column_name] = new_value
    return dataframe


def normalize_image(img: np.ndarray, epsilon: float = 1e-8) -> np.ndarray:
    img_min, img_max = img.min(), img.max()
    img_norm = (img - img_min) / (img_max - img_min + epsilon) * 255
    return img_norm.astype(np.uint8)

# # Função para combinar uma lista de imagens PIL
# def combine_images_vertically(pil_images):
#     combined_width = max([img.width for img in pil_images])
#     combined_height = sum([img.height for img in pil_images])

#     combined_image = Image.new('RGB', (combined_width, combined_height))

#     y_offset = 0
#     for img in pil_images:
#         combined_image.paste(img, (0, y_offset))
#         y_offset += img.height

#     return combined_image

def combine_images_vertically(pil_images: List[Image.Image]) -> Image.Image:
    combined_width = max([img.width for img in pil_images])
    combined_height = sum([img.height for img in pil_images])

    combined_image = Image.new('RGB', (combined_width, combined_height))

    y_offset = 0
    for img in pil_images:
        img = img.convert('RGB')  # Certifique-se de que a imagem está no modo RGB
        combined_image.paste(img, (0, y_offset))
        y_offset += img.height

    return combined_image

def get_dicom_header_info(dicom_file, tag):
    """
    Função para extrair uma informação específica do cabeçalho DICOM.

    Args:
        dicom_file (str): Caminho para o arquivo DICOM.
        tag (tuple): Tag DICOM no formato (grupo, elemento), por exemplo, (0x0010, 0x0010) para o nome do paciente.

    Returns:
        O valor associado à tag DICOM ou None se a tag não for encontrada no arquivo.
    """
    ds = pydicom.dcmread(dicom_file)

    if tag in ds:
        return ds[tag].value
    else:
        return None

def dicom_metadata_to_dataframe(file_path):
    # Ler o arquivo DICOM
    dicom_file = pydicom.dcmread(file_path)

    # Criar um dicionário vazio para armazenar os metadados do cabeçalho DICOM
    metadata_dict = {}

    # Iterar sobre os elementos do arquivo DICOM
    for data_element in dicom_file:
        # Obter a tag, o nome da tag (keyword) e o valor do elemento
        tag = str(data_element.tag)
        keyword = pydicom.datadict.keyword_for_tag(data_element.tag)
        value = data_element.value

        # Armazenar o valor no dicionário usando a tag e a keyword como chave
        metadata_dict[f"{tag} ({keyword})"] = value

    # Converter o dicionário em um DataFrame
    metadata_df = pd.DataFrame.from_dict(metadata_dict, orient='index', columns=['Value'])

    return metadata_df

def extrair_info_apos_palavra(texto, palavra_chave):
    regex_pattern = r'(?<=' + re.escape(palavra_chave) + r')\s*([\d.]+)'
    numeros = re.findall(regex_pattern, texto)

    if numeros:
        # Retorna o primeiro número encontrado após a palavra-chave
        return numeros[0]
    else:
        return None

def extrair_informacao(texto, string1, string2):
    try:
        inicio = texto.find(string1) + len(string1)
        final = texto.find(string2, inicio)
        resultado = texto[inicio:final].strip()
        return resultado
    except Exception as e:
        print(f"Erro ao extrair informação: {e}")
        return None

def converter_coluna_data(coluna):
    formato_entrada = "%d-%b-%y %H:%M:%S"
    formato_saida = "%d/%m/%Y"

    # Aplica a conversão a cada elemento da coluna e retorna a nova coluna
    return coluna.apply(lambda data_str: datetime.strptime(data_str, formato_entrada).strftime(formato_saida))

def renomear_arquivos(diretorio):
    arquivos = [f for f in os.listdir(diretorio) if os.path.isfile(os.path.join(diretorio, f))]

    # Ordena os arquivos usando o nome completo do arquivo
    arquivos.sort(key=lambda x: (os.path.splitext(x)[0], os.path.splitext(x)[1]))

    # Primeiro passo: renomear todos os arquivos para nomes temporários
    arquivos_temporarios = []
    for i, arquivo in enumerate(arquivos):
        nome_antigo = os.path.join(diretorio, arquivo)
        extensao = os.path.splitext(arquivo)[1]
        nome_temporario = os.path.join(diretorio, f"tmp_{i}{extensao}")

        os.rename(nome_antigo, nome_temporario)
        arquivos_temporarios.append(nome_temporario)

    # Segundo passo: renomear os arquivos temporários para a sequência correta
    for i, arquivo_temp in enumerate(arquivos_temporarios):
        nome_antigo = arquivo_temp
        extensao = os.path.splitext(arquivo_temp)[1]
        nome_novo = os.path.join(diretorio, f"{i}{extensao}")

        os.rename(nome_antigo, nome_novo)
        print(f"Arquivo '{nome_antigo}' renomeado para '{nome_novo}'")

def extrair_minutos(string):
    # Define a expressão regular para encontrar um número seguido de "min"
    pattern = r"(\d+\.?\d*)min"

    # Procura na string pela expressão regular
    match = re.search(pattern, string)

    # Se encontrar a expressão regular, retorna o número encontrado
    if match:
        return float(match.group(1))
    else:
        return None

def extrair_dose(string):
    # Define a expressão regular para encontrar um número seguido de "min"
    pattern = r"(\d+\.?\d*)mGy"

    # Procura na string pela expressão regular
    match = re.search(pattern, string)

    # Se encontrar a expressão regular, retorna o número encontrado
    if match:
        return float(match.group(1))
    else:
        return None

def mostra_botoes ():
  display(ano_escolhido, mes_escolhido,semana_escolhida)
  return

def vetores_para_dataframe(*vetores, colunas=None):
    # Criando um dicionário vazio para armazenar os vetores como colunas
    colunas_dict = {}

    # Verificando se os nomes das colunas foram fornecidos, caso contrário, usar nomes padrão
    if colunas is None:
        colunas = [f'coluna_{i}' for i in range(len(vetores))]

    # Adicionando cada vetor como uma coluna no dicionário
    for nome_coluna, vetor in zip(colunas, vetores):
        colunas_dict[nome_coluna] = vetor

    # Criando um DataFrame a partir do dicionário de colunas
    dataframe = pd.DataFrame(colunas_dict)

    return dataframe

def combine_images_vertically_cv(images):
    images_np = [np.array(img) for img in images]
    return cv2.vconcat(images_np)

def mostra_botoes_2 ():
  display(linha_escolhida, coluna_escolhida, valor_novo)
  return

In [3]:
ano_escolhido = widgets.ToggleButtons(
    options=['2021','2022', '2023', '2024'],
    description='Ano',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)


mes_escolhido = widgets.ToggleButtons(
    options=['janeiro','fevereiro','marco','abril','maio','junho','julho','agosto','setembro','outubro','novembro','dezembro'],
    description='Mes:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

semana_escolhida = widgets.ToggleButtons(
    options=['1','2','3','4','5'],
    description='Semana',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

linha_escolhida = widgets.IntText(
    value = 0,
    description='Linha: ',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

coluna_escolhida = widgets.ToggleButtons(
    options=['Dose (mGy)','KPA (uGym2)','Tempo (min)','Num Exposicoes'],
    description='Coluna de interesse: ',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

valor_novo = widgets.IntText(
    value = 0,
    description='Valor novo: ',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

# Choose the period:
---



In [4]:
mostra_botoes ()

ToggleButtons(button_style='success', description='Ano', options=('2021', '2022', '2023', '2024'), tooltips=('…

ToggleButtons(button_style='success', description='Mes:', options=('janeiro', 'fevereiro', 'marco', 'abril', '…

ToggleButtons(button_style='success', description='Semana', options=('1', '2', '3', '4', '5'), tooltips=('Desc…

# Read DCM files from your Personal Drive

In [8]:
mes = mes_escolhido.value
numero_semana = semana_escolhida.value
ano = ano_escolhido.value

path_dcm = '/content/drive/MyDrive/dados_da_hemo/{a}/{b}/semana{c}/'.format(a=ano,b=mes,c=numero_semana)
imagem_path = '/content/drive/MyDrive/dados_da_hemo/{a}/{b}/semana{c}/imagens'.format(a=ano,b=mes,c=numero_semana)

In [14]:
#renomear_arquivos(path_dcm)

## Retirando as informações do cabeçalho DICOM dos procedimentos

In [ ]:
tag_data = (0x0008, 0x0023)
tag_id_paciente = (0x0010,0x0020)
tag_sexo_paciente = (0x0010,0x0040)
tag_idade_paciente = (0x0010,0x1010)
tag_tipo_procedimento = (0x0040,0x0254)
tag_medicos = (0x0008,0x1050)

data = []
id_paciente = []
sexo_paciente = []
idade_paciente = []
tipo_procedimento = []
medicos = []

In [ ]:
arquivos = [f for f in os.listdir(path_dcm) if os.path.isfile(os.path.join(path_dcm, f))]

# Conta o número de arquivos
numero_de_arquivos = len(arquivos)

for i in range (numero_de_arquivos):
  arquivo_dicom = '/content/drive/MyDrive/dados_da_hemo/{a}/{b}/semana{c}/{d}.dcm'.format(a=ano,b=mes,c=numero_semana,d=i)
  data_desse_paciente = get_dicom_header_info(arquivo_dicom, tag_data)
  id_desse_paciente = get_dicom_header_info(arquivo_dicom, tag_id_paciente)
  sexo_desse_paciente = get_dicom_header_info(arquivo_dicom, tag_sexo_paciente)
  idade_desse_paciente = get_dicom_header_info(arquivo_dicom, tag_idade_paciente)
  tipo_desse_procedimento = get_dicom_header_info(arquivo_dicom, tag_tipo_procedimento)
  medicos_desse_procedimento =  get_dicom_header_info(arquivo_dicom, tag_medicos)

  data.append(data_desse_paciente)
  id_paciente.append(id_desse_paciente)
  sexo_paciente.append(sexo_desse_paciente)
  idade_paciente.append(idade_desse_paciente)
  tipo_procedimento.append(tipo_desse_procedimento)
  medicos.append(medicos_desse_procedimento)

In [ ]:
nomes_das_colunas = ['ID','Data','Sexo Paciente','Idade Paciente','Tipo Procedimento','Medicos']
dataframe_cabecalho_dcm = vetores_para_dataframe(id_paciente,data,sexo_paciente,idade_paciente, tipo_procedimento,medicos,colunas=nomes_das_colunas)
dataframe_cabecalho_dcm['ID'] = dataframe_cabecalho_dcm['ID'].astype(int)
dataframe_cabecalho_dcm

\# Parte 2: Transformando as imagens .dcm em imagens .png e salvando no drive

In [ ]:
imagens = []

# def show_slices_from_dicom_file(dicom_file):
#     # Lê o arquivo DICOM
#     ds = pydicom.dcmread(dicom_file)

#     if ds.pixel_array.ndim == 2:
#         slices = [ds.pixel_array.astype(np.uint8)]
#     elif ds.pixel_array.ndim == 3:
#         slices = [ds.pixel_array[i].astype(np.uint8) for i in range(ds.pixel_array.shape[0])]
#     else:
#         print(f"O arquivo DICOM não possui dados de imagem no formato esperado (1024 x 1024).")
#         return

#     # Remove as bordas pretas dos slices
#     cropped_slices = [remove_black_borders(slice_img) for slice_img in slices]

#     # Junta os slices verticalmente em uma única imagem
#     combined_image = np.vstack(cropped_slices)



#     # Adiciona a imagem na lista de imagens
#     imagens.append(combined_image)

def remove_black_borders(image):
    mask = image > 0
    coords = np.argwhere(mask)
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0) + 1
    return image[y_min:y_max, x_min:x_max]

def save_slices_as_png(dicom_file):
    # Lê o arquivo DICOM
    ds = pydicom.dcmread(dicom_file)

    if ds.pixel_array.ndim == 2:
        slices = [ds.pixel_array.astype(np.uint8)]
    elif ds.pixel_array.ndim == 3:
        slices = [ds.pixel_array[i].astype(np.uint8) for i in range(ds.pixel_array.shape[0])]
    else:
        print(f"O arquivo DICOM não possui dados de imagem no formato esperado (1024 x 1024).")
        return

    # Remove as bordas pretas dos slices
    cropped_slices = [remove_black_borders(slice_img) for slice_img in slices]

    # Junta os slices verticalmente em uma única imagem
    combined_image = np.vstack(cropped_slices)
    combined_image = Image.fromarray(combined_image)

    # Salva a imagem em um arquivo PNG
    imagens.append(combined_image)

In [ ]:
for i in range (numero_de_arquivos):
  dicom_file = '/content/drive/MyDrive/dados_da_hemo/{a}/{b}/semana{c}/{d}.dcm'.format(a=ano,b=mes,c=numero_semana,d=i)
  save_slices_as_png(dicom_file)

for i in range(numero_de_arquivos):
    image = imagens[i]
    image.save(f"{imagem_path}/imagem{i}.png", quality=95)

## Manipulando as imagens e retirando as informações do texto

In [ ]:
texto = []

for i in range(numero_de_arquivos):
  try:
    txt = pytesseract.image_to_string(Image.open('/content/drive/MyDrive/dados_da_hemo/{a}/{b}/semana{c}/imagens/imagem{d}.png'.format(a=ano,b=mes,c=numero_semana,d=i)))
    texto.append(txt)
  except:
    txt =  'erro'
    texto.append(txt)
    print (i)

In [ ]:
print (texto[0])

## Retira apenas a porção final das imagens e tira o espaço entre os números

In [ ]:
texto_cortado = []
texto_cortado_espaco = []

def extract_exposure_data(text):
    pattern = r'(?<=\*\*\*accumulated exposure data\*\*\*).*'
    result = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    if result:
        return result.group(0).strip()
    else:
        return ''

def remove_spaces_between_numbers(text):
    # Procura por um padrão de espaços entre números (exemplo: "1234.   8")
    pattern = r'(\d+)\s*([.,])?\s*(\d+)?'

    # Substitui o padrão pela sequência de números sem espaços
    return re.sub(pattern, r'\1\2\3', text)

for i in range (numero_de_arquivos):
  texto_cortado.append(extract_exposure_data(texto[i]))
  texto_cortado_espaco.append(remove_spaces_between_numbers(texto_cortado[i]))

In [ ]:
texto_cortado_espaco[0]

In [ ]:
numeros_do_texto = []
dose_mgy = []
kpa_uGym2 = []
tempo_fluoro = []
num_exp = []

def extrair_numero_exposures(texto):
    # Cria a expressão regular para encontrar o padrão "Exposures:" seguido de um número (com ou sem espaços e caracteres adicionais)
    pattern = r'Exposures:[^0-9]*([\d.]+)'

    # Procura pelo padrão no texto
    match = re.search(pattern, texto, flags=re.IGNORECASE)

    # Se o padrão for encontrado, retorna o número extraído
    if match:
        return float(match.group(1))
    else:
        return None

def extrair_numero_fluoro(texto):
    # Cria a expressão regular para encontrar o padrão "Fluoro", "Flucro" ou "Fluore" seguido de um número (com ou sem espaços e caracteres adicionais)
    pattern = r'(Fluoro|Flucro|Fluore)[^0-9]*([\d.]+)'

    # Procura pelo padrão no texto
    match = re.search(pattern, texto, flags=re.IGNORECASE)

    # Se o padrão for encontrado, retorna o número extraído
    if match:
        return float(match.group(2))
    else:
        return None

def extract_numbers(text):
    numbers = re.findall(r'\d+(?:\.\d+)?', text)
    return [float(n) for n in numbers]

for i in range (numero_de_arquivos):
    a = extract_numbers(texto_cortado_espaco[i])

    # Se a lista 'a' não tiver pelo menos dois elementos, continue com a próxima iteração
    if len(a) < 2:
        continue

    dose_da_vez = a[-1]
    kpa_uGym2_da_vez = a[-2]
    tempo_fluoro.append(extrair_numero_fluoro(texto_cortado_espaco[i]))
    num_exp.append(extrair_numero_exposures(texto_cortado_espaco[i]))
    dose_mgy.append(dose_da_vez)
    kpa_uGym2.append(kpa_uGym2_da_vez)

In [ ]:
def vetor_para_coluna_dataframe(vetor, nome_coluna):
    # cria um dataframe com uma coluna e os valores do vetor
    df = pd.DataFrame({nome_coluna: vetor})
    return df

dataframe_dose = vetor_para_coluna_dataframe(dose_mgy, 'Dose (mGy)')
dataframe_kpa = vetor_para_coluna_dataframe(kpa_uGym2, 'KPA (uGym2)')
dataframe_tempo = vetor_para_coluna_dataframe(tempo_fluoro, 'Tempo (min)')
dataframe_exp = vetor_para_coluna_dataframe(num_exp, 'Num Exposicoes')

## Criando o dataframe final

### Juntando os dataframes dcm com dose e kpa

In [ ]:
df_concat = pd.concat([dataframe_cabecalho_dcm, dataframe_dose, dataframe_kpa, dataframe_tempo, dataframe_exp], axis=1)
df_concat

### Arrumando as datas e transformando valores numéricos em variáveis númericas

In [ ]:
from datetime import datetime

def format_date(column):
    new_dates = []
    for date in column:
        if len(date) == 8:
            new_date = datetime.strptime(date, '%Y%m%d').strftime('%d/%m/%Y')
        elif len(date) == 10:
            new_date = datetime.strptime(date, '%d/%m/%Y').strftime('%d/%m/%Y')
        else:
            new_date = date  # leave unchanged if not in expected format
        new_dates.append(new_date)
    return new_dates

df_concat['Data'] = format_date(df_concat['Data'])

# Filter the Dataframe

In [5]:
mostra_botoes_2 ()

IntText(value=0, description='Linha: ')

ToggleButtons(button_style='success', description='Coluna de interesse: ', options=('Dose (mGy)', 'KPA (uGym2)…

IntText(value=0, description='Valor novo: ')

In [18]:
# replace_value (df_concat, linha_escolhida.value, coluna_escolhida.value, valor_novo.value)
# df_concat

In [19]:
# def remove_virgulas(medico):
#     return ''.join([c for c in medico if c != ','])

# # Aplicar a função na coluna 'Medicos'
df_concat['Medicos'] = df_concat['Medicos'].apply(remove_virgulas)

df_concat['KPA (uGym2)'] = df_concat['KPA (uGym2)'].astype(int)
df_concat['Dose (mGy)'] = df_concat['Dose (mGy)'].astype(int)
df_concat['Tempo (min)'] = df_concat['Tempo (min)'].astype(float)
df_concat['ID'] = df_concat['ID'].astype(int)

df_concat['Dose (mGy)'] = df_concat['Dose (mGy)'].astype(float)
df_concat['Taxa de Dose (mGy/s)'] = round(df_concat['Dose (mGy)']/(60*df_concat['Tempo (min)']),1)
df_concat

### Salve seu dataframe como uma tabela do excel

In [ ]:
def salvar_dataframe_excel(df, nome_arquivo, diretorio):
    # Concatena o diretório e o nome do arquivo para criar o caminho completo
    caminho_arquivo = f"{diretorio}/{nome_arquivo}.xlsx"

    # Salva o DataFrame como um arquivo Excel no caminho especificado
    df.to_excel(caminho_arquivo, index=False, engine='openpyxl')

nome_arquivo = 'dados{a}_{b}_{c}'.format(a=mes,b=numero_semana,c=ano)
diretorio_arquivo = '/content/drive/MyDrive/dados_da_hemo/{a}/tabelas_excel/'.format(a=ano,b=mes)
salvar_dataframe_excel(df_concat, nome_arquivo, diretorio_arquivo)